A tutorial based on [An Algorithmic Introduction to Numerical Simulation of Stochastic Differential Equations][1] by Desmond J. Higham.

[1]: https://doi.org/10.1137/S0036144500378302

In [ ]:
%matplotlib notebook

import numpy as np
import numpy.random as random
import matplotlib.pyplot as plt

import sde
random.seed(0)

# Brownian Motion

Definition of Brownian motion:

1.  $W(0) = 0 \quad .$
2.  $W(t) - W(s) \sim \mathcal{N}(0, \sqrt{t-s} \quad , \quad 0 \leq s < t \leq T \quad .$
3.  $W(t) - W(s)$ and $W(v) - W(u)$, $0 \leq s < t < u < v \leq T$, are independent.

Example:

In [ ]:
T = 1.
N = 500
t = np.linspace(0., 1., N + 1)

N_ens = 100
W_ens, dW_ens = sde.brown(T, N, N_ens)

fig = plt.figure(1)
ax = fig.gca()
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$W$")
for k in range(N_ens):
    ax.plot(t, W_ens[k, :])
    
fig = plt.figure(2)
ax = fig.gca()
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"standard deviation of $W$")
std_dev = np.sqrt(np.sum(W_ens**2, axis=0) / N_ens)
ax.plot(t, np.sqrt(t), label="analytical")
ax.plot(t, std_dev, label="numerical")
ax.legend()

# Stochastic Integrals

We are going to approximate the stochastic integral $\int_0^T{h(t)\,\mathrm{d}W(t)}$ by Riemann sums.
Inside a stochastic differential equation, the term $h(t)\,\mathrm{d}W(t)$ may represent stochastic forcing with Gaussian noise of time-dependent amplitude $h(t)$.
The representation of a stochastic integral by a Riemann sum is not unique.
Unlike for ordinary differential equations, the representations are not equivalent as $\delta t \to 0$.

## Itô Integral

The Itô integral is defined as the left-hand sum:
\begin{equation}
\int_0^T{h(t)\,\mathrm{d}W(t)} = \lim_{N \to \infty}{\sum_{i=0}^{N-1}{h(t_i)\left(W(t_{i+1}) - W(t_i)\right)}}
\end{equation}

For $h(t) = W(t)$, the Itô integral gives:
\begin{align}
\sum_{i=0}^{N-1}{W(t_i)\left(W(t_{i+1}) - W(t_i)\right)}
&= \frac{1}{2}\sum_{i=0}^{N-1}\left[W(t_{i+1})^2 - W(t_i)^2 - \left(W(t_{i+1}) - W(t_i)\right)^2\right] \\
&= \frac{1}{2}\sum_{i=0}^{N-1}\left[W(t_{i+1})^2 - W(t_i)^2\right] - \frac{1}{2}\sum_{i=0}^{N-1}\left(W(t_{i+1}) - W(t_i)\right)^2 \\
&= \frac{1}{2}\left[W(T)^2 - W(0)^2\right] - \frac{1}{2}\sum_{i=0}^{N-1}\left(W(t_{i+1}) - W(t_i)\right)^2
\end{align}
\begin{equation}
\lim_{N \to \infty}{\sum_{i=0}^{N-1}{W(t_i)\left(W(t_{i+1}) - W(t_i)\right)}} = \frac{1}{2}\left[W(T)^2 - W(0)^2\right] - \frac{T}{2}
\end{equation}

Example:

In [ ]:
ito = np.sum(W_ens[:, :-1] * dW_ens, axis=1)

fig = plt.figure(3)
ax = fig.gca()
ax.set_title("Itô integral")
ax.set_xlabel(r"$W(T)$")
ax.set_ylabel(r"$\int_0^T{W(t)\,\mathrm{d}W(t)}$")
ax.plot(np.sort(W_ens[:, -1]), 0.5 * np.sort(W_ens[:, -1])**2 - 0.5 * T, label="analytical")
ax.scatter(W_ens[:, -1], ito, label="numerical")
ax.legend()

## Stratonovich Integral

The Stratonovich integral is defined as the mid-point sum:
\begin{equation}
\int_0^T{h(t)\,\mathrm{d}W(t)} = \lim_{N \to \infty}{\sum_{i=0}^{N-1}{h\left(\frac{t_i + t_{i+1}}{2}\right)\left(W(t_{i+1}) - W(t_i)\right)}}
\end{equation}

The conditional probability distribution of $W\left(\frac{t_i + t_{i+1}}{2}\right)$ given $W(t_i)$ and $W(t_{i+1})$ is given by
\begin{equation}
W\left(\frac{t_i + t_{i+1}}{2}\right) = \frac{W(t_i) + W(t_{i+1})}{2} + \Delta Z \quad , \quad \Delta Z \sim \mathcal{N}\left(0, \frac{\delta t}{4}\right) \quad .
\end{equation}
For $h(t) = W(t)$, the Stratonovich integral gives:
\begin{align}
\sum_{i=0}^{N-1}{W\left(\frac{t_i + t_{i+1}}{2}\right)\left(W(t_{i+1}) - W(t_i)\right)}
&= \sum_{i=0}^{N-1}{\left[\frac{W(t_i) + W(t_{i+1})}{2} + \Delta Z\right]\left(W(t_{i+1}) - W(t_i)\right)} \\
&= \frac{1}{2}\sum_{i=0}^{N-1}\left[W(t_{i+1})^2 - W(t_i)^2\right] + \Delta Z\sum_{i=0}^{N-1}\left(W(t_{i+1}) - W(t_i)\right) \\
&= \frac{1}{2}\left[W(T)^2 - W(0)^2\right] + \Delta Z\left(W(T) - W(0)\right)
\end{align}
\begin{equation}
\lim_{N \to \infty}{\sum_{i=0}^{N-1}{W\left(\frac{t_i + t_{i+1}}{2}\right)\left(W(t_{i+1}) - W(t_i)\right)}} = \frac{1}{2}\left[W(T)^2 - W(0)^2\right]
\end{equation}

Example:

In [ ]:
strat = np.sum(0.5 * (W_ens[:, :-1] + W_ens[:, 1:]) * dW_ens, axis=1) + random.normal(0, 0.25 * T / N, N_ens)

fig = plt.figure(4)
ax = fig.gca()
ax.set_title("Stratonovich integral")
ax.set_xlabel(r"$W(T)$")
ax.set_ylabel(r"$\int_0^T{W(t)\,\mathrm{d}W(t)}$")
ax.plot(np.sort(W_ens[:, -1]), 0.5 * np.sort(W_ens[:, -1])**2, label="analytical")
ax.scatter(W_ens[:, -1], strat, label="numerical")
ax.legend()

# The Euler-Maruyama Method

# Strong and Weak Convergence of the Euler-Maruyama Method